## Code to generate mr_proc_manifest.csv
- Based on this [schema](https://www.neurobagel.org/documentation/mr_proc/configs/)

### QPN naming convention
- `visit`: timepoint of **ANY** clinical (i.e. UPDRS, MoCA, Neuropsy) data collected 
    - e.g. V01, V02 etc. 
- `session`: timepoint of MRI collection
    - e.g. ses-01, ses-02 etc.
- `event`: timepoint relative to a consensus baseline (used for inter-modality i.e. MRI vs clinical data harmonization) 
    - e.g. baseline, m06, m12 etc. 

In [1]:
import pandas as pd
import numpy as np

### Paths

In [11]:
releases_dir = "/home/nikhil/projects/Parkinsons/qpn/releases/"

previous_release = "Apr_2022"
current_release = "Apr_2022"

# Previous mr_proc manifest
previous_mr_proc_manifest_csv = f"{releases_dir}{previous_release}/tabular/mr_proc_manifest.csv"

# Current mr_proc manifest
current_mr_proc_manifest_csv = f"{releases_dir}{current_release}/tabular/mr_proc_manifest.csv"

# Current recruit manifest
current_recruit_manifest_xls = f"{releases_dir}{current_release}/tabular/QPN_Imaging_Codes.xlsx"

### Read recruitment manifest from previous release

In [4]:
previous_recruit_manifest_df = pd.read_csv(previous_mr_proc_manifest_csv)
previous_recruit_manifest_df["participant_id"] = previous_recruit_manifest_df["participant_id"].str.strip()
mr_proc_participants_previous = previous_recruit_manifest_df["participant_id"].dropna().unique()
n_mr_proc_participants_previous = len(mr_proc_participants_previous)

print(f"number of participants from previous mr_proc release: {n_mr_proc_participants_previous}")
previous_recruit_manifest_df.head()

number of participants from previous mr_proc release: 208


,participant_id,visit,session,datatype,participant_dicom_dir
0,MNI0056,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']",MNI0056_864854_MRI01_MNI_20210818_151510608
1,MNI0058,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']",MNI0058_197308_MRI01_MNI_20210818_105219098
2,MNI0068,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']",MNI0068_842090_MRI01_MNI_20210827_150412426
3,MNI0079,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']",MNI0079_760662_MRI01_MNI_20211222_134225878
4,MNI0103,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']",MNI0103_369057_MRI01_MNI_20211116_132143505


In [10]:
missing_dicom_ids = previous_recruit_manifest_df[(previous_recruit_manifest_df["participant_dicom_dir"].isna()) & 
                             (previous_recruit_manifest_df["session"] == "ses-01")]["participant_id"].values
missing_dicom_ids

array(['MNI0149', 'MNI0224', 'MNI0326', 'PD00472', 'PD00576', 'PD00583',
       'PD00620', 'PD00792', 'PD00849', 'PD00911', 'PD01067', 'PD01071',
       'PD01126', 'PD01134', 'PD01161', 'PD01165', 'PD01201', 'PD01232',
       'PD01237', 'PD01247', 'PD01340', 'PD01500', 'PD01590'],
      dtype=object)

### Read latest recruitment manifest
- Using group as a longitudinal clinical var 
- This allows inclusion of participants with demographic info but without specific clinical scores

In [12]:
sheet_name = "Dashboard" #"recruit_manifest"

assessment_dict = {
    "MRI": ["MRI"],
    "clinical": ["Neuropsy","TUG","UPDRS","MoCA"] 
}

# useful cols
useful_cols = {"MRI Time 1": "MRI_V1", "MRI Time 2": "MRI_V2", 
            "Neuropsy Time 1":"Neuropsy_V1", "Neuropsy Time 2":"Neuropsy_V2", 
            "TUG Time 1": "TUG_V1", "TUG Time 2": "TUG_V2",
            "UPDRS (part III) Time 1":"UPDRS_V1", "UPDRS (part III) Time 2":"UPDRS_V2",
            "MoCA Time 1":"MoCA_V1", "MoCA Time 2":"MoCA_V2","MoCA time 3":"MoCA_V3"}

column_list = ["participant_id", "group"] + list(useful_cols.keys())

dashboard_df = pd.read_excel(current_recruit_manifest_xls,sheet_name=sheet_name, engine='openpyxl',usecols=column_list)

dashboard_df = dashboard_df.dropna(how="all", axis=0)
dashboard_df["participant_id"] = dashboard_df["participant_id"].str.strip()

dashboard_df = dashboard_df.rename(columns=useful_cols)
for col in useful_cols.values():
    dashboard_df[col] = pd.to_datetime(dashboard_df[col],errors="coerce", dayfirst=True) #Note: format="%d-%m-%Y" skips MRI02 dates
    
mr_proc_participants_current = dashboard_df["participant_id"].dropna().unique()
n_mr_proc_participants_current = len(mr_proc_participants_current)
print(f"number of participants from current mr_proc release: {n_mr_proc_participants_current}")
n_MRI01 = len(dashboard_df[~dashboard_df["MRI_V1"].isna()])
n_MRI02 = len(dashboard_df[~dashboard_df["MRI_V2"].isna()])
print(f"n_MRI01: {n_MRI01}, n_MRI02: {n_MRI02}")
dashboard_df.head()

number of participants from current mr_proc release: 208
n_MRI01: 208, n_MRI02: 16


,participant_id,group,MRI_V1,MRI_V2,Neuropsy_V1,Neuropsy_V2,UPDRS_V1,UPDRS_V2,MoCA_V1,MoCA_V2,MoCA_V3,TUG_V1,TUG_V2
0,MNI0056,PD,2021-08-18,NaT,2021-07-30,NaT,2021-06-11,NaT,2021-06-11 00:00:00,NaN,NaT,2021-09-03,NaT
1,MNI0058,PD,2021-08-18,NaT,2021-08-18,NaT,2021-07-23,NaT,2021-07-23 00:00:00,NaN,NaT,2021-07-23,NaT
2,MNI0068,PD,2021-08-27,NaT,2021-08-18,NaT,2021-08-27,NaT,NaN,NaN,NaT,NaT,NaT
3,MNI0079,PD,2021-12-22,NaT,2022-01-21,NaT,2022-01-21,NaT,2021-12-22 00:00:00,NaN,NaT,2021-07-02,NaT
4,MNI0103,PD,2021-11-08,NaT,2021-10-01,NaT,2021-10-01,NaT,2021-11-08 00:00:00,NaN,NaT,NaT,NaT


In [14]:
dashboard_df[dashboard_df["participant_id"].isin(missing_dicom_ids)][["participant_id","MRI_V1"]]

,participant_id,MRI_V1
12,MNI0149,2022-02-15
21,MNI0224,2022-03-16
28,MNI0326,2022-06-10
48,PD00472,2019-06-21
51,PD00576,2019-04-10
52,PD00583,2019-09-04
53,PD00620,2019-06-20
64,PD00792,2019-06-20
66,PD00849,2019-05-27
70,PD00911,2019-08-28


### Get new participants

In [12]:
partcipants_additions = list(set(mr_proc_participants_current) - set(mr_proc_participants_previous))

print(f"number of new participants: {len(partcipants_additions)}")

number of new participants: 2


### Generate current_mr_proc_manifest_df

- Populate manifest with available clinical visits / MRI sessions
- Datatypes: Assuming QPN has all 4 BIDS datatypes: ["anat","dwi","fmap","func"]

- Sample `mr_proc_manifest.csv`

| participant_id | participant_dicom_dir | visit | session | datatype                     | bids_id |
|----------------|-----------------------|-------|---------|------------------------------|---------|
| 001            | MyStudy_001_V2021      | V01   | ses-01  | ["anat","dwi","fmap","func"] | sub-001 |
| 001            | MyStudy_001_V2022      | V02   | ses-02  | ["anat"]                     | sub-001 |
| 002            | MyStudy_002_V2021      | V01   | ses-01  | ["anat","dwi"]               | sub-002 |
| 002            | MyStudy_002_V2024      | V03   | ses-03  | ["anat","dwi"]               | sub-002 |

In [13]:
visit_ids = ["1","2"]
manifest_cols = ["visit","session","datatype"]
avail_datatypes = ["anat","dwi","fmap","func"]

current_mr_proc_manifest_df = pd.DataFrame()

for visit_id in visit_ids:
    _df = pd.DataFrame(index=mr_proc_participants_current, columns=manifest_cols)
    for k,v in assessment_dict.items():
        assessment_cols = [f"{vx}_V{visit_id}" for vx in v]
        avail_ids = dashboard_df[~(dashboard_df[assessment_cols].isna().all(axis=1))]["participant_id"]

        visit_label = f"V0{visit_id}"  
        _df.loc[avail_ids,"visit"] = visit_label  

        if k == "MRI":    
            time_col = "session"    
            visit_label = f"ses-0{visit_id}"
            _df.loc[avail_ids,time_col] = visit_label
            
            for pid in avail_ids:
                _df.loc[pid,"datatype"] = avail_datatypes

    current_mr_proc_manifest_df = current_mr_proc_manifest_df.append(_df)

# Remove participants with all missing visits/session
current_mr_proc_manifest_df = current_mr_proc_manifest_df.dropna(axis=0, how="all")
current_mr_proc_manifest_df.index = current_mr_proc_manifest_df.index.str.strip()
current_mr_proc_manifest_df.head()

,visit,session,datatype
MNI0056,V01,ses-01,"[anat, dwi, fmap, func]"
MNI0058,V01,ses-01,"[anat, dwi, fmap, func]"
MNI0068,V01,ses-01,"[anat, dwi, fmap, func]"
MNI0079,V01,ses-01,"[anat, dwi, fmap, func]"
MNI0103,V01,ses-01,"[anat, dwi, fmap, func]"


### Save update CSV

In [15]:
save_current_mr_proc_manifest = False
if save_current_mr_proc_manifest:
    print(f"Saving new mr_proc manifest here: {current_mr_proc_manifest_csv}")
    current_mr_proc_manifest_df = current_mr_proc_manifest_df.reset_index().rename(columns={"index":"participant_id"})
    current_mr_proc_manifest_df.to_csv(current_mr_proc_manifest_csv,index=None)

Saving new mr_proc manifest here: /home/nikhil/projects/Parkinsons/qpn/releases/Apr_2022/tabular/mr_proc_manifest.csv
